## 0.0 Setup

In [307]:
import math
import corona
import pandas as pd

In [308]:
backhowmanyrows = 0
ENDFLAG = 0
DFLAG = 0
DVFLAG = 0
NTN = 0
NBN = 0
NN = 0
VRMAX = 0
peakvs = 0
peakvp = 0
tc5 = 1

In [309]:
# Defining parameters for specific devices
print("Please select the device:")
print("1. UNUICTPPFF Machine")
print("2. NX2-RGNeon Machine")
print("3. PF1000 27D Machine")
print("4. Custom Parameters")
print("5. Custom Parameters")

device = int(input("Enter the number corresponding to your device: "))

if device == 1:
    L0, C0, RADB, RADA, Z0, R0, V0, P0, MW, ZN, dissociatenumber, massf, CURRF, massfr, currfr = 110.0, 30.0, 3.2, 0.95, 16.0, 12.0, 15.0, 3.5, 4.0, 1.0, 2.0, 0.08, 0.7, 0.16, 0.7
elif device == 2:
    L0, C0, RADB, RADA, Z0, R0, V0, P0, MW, ZN, dissociatenumber, massf, CURRF, massfr, currfr = 20.0, 28.0, 4.1, 1.9, 5.0, 2.3, 11.0, 2.63, 20.0, 10.0, 1.0, 0.0635, 0.7, 0.16, 0.7
elif device == 3:
    L0, C0, RADB, RADA, Z0, R0, V0, P0, MW, ZN, dissociatenumber, massf, CURRF, massfr, currfr = 33.0, 1332.0, 16.0, 11.55, 60.0, 6.3, 27.0, 3.5, 4.0, 1.0, 2.0, 0.14, 0.7, 0.35, 0.7
elif device == 4:
    L0, C0, RADB, RADA, Z0, R0, V0, P0, MW, ZN, dissociatenumber, massf, CURRF, massfr, currfr = 60.0, 120.0, 12.5, 7.5, 30.0, 15.0, 100.0, 1.0, 4.0, 1.0, 2.0, 0.0635, 0.7, 0.25, 0.7
elif device == 5:
    L0 = float(input("Enter L0: "))
    C0 = float(input("Enter C0: "))
    RADB = float(input("Enter RADB: "))
    RADA = float(input("Enter RADA: "))
    Z0 = float(input("Enter Z0: "))
    R0 = float(input("Enter R0: "))
    V0 = float(input("Enter V0: "))
    P0 = float(input("Enter P0: "))
    MW = float(input("Enter MW: "))
    ZN = float(input("Enter ZN: "))
    dissociatenumber = float(input("Enter dissociatenumber: "))
    massf = float(input("Enter massf: "))
    CURRF = float(input("Enter CURRF: "))
    massfr = float(input("Enter massfr: "))
    currfr = float(input("Enter currfr: "))
else:
    print("Invalid input. Please enter a number between 1 and 5c.")


Please select the device:
1. UNUICTPPFF Machine
2. NX2-RGNeon Machine
3. PF1000 27D Machine
4. Custom Parameters
5. Custom Parameters
Enter the number corresponding to your device: 4


In [310]:
# Defining parameters for specific devices
print("Please select the device:")
print("1. Set device")
print("2. Custom device")

device = int(input("Enter the number corresponding to your device: "))

if device == 1:
    fen, radratiolimit, rctimelimit = 0.14, 0.04, 1.0
elif device == 2:
    fen = float(input("Enter fen: "))
    radratiolimit = float(input("Enter radratiolimit: "))
    rctimelimit = float(input("Enter rctimelimit: "))
else:
    print("Invalid input. Please enter a number between 1 and 2.")



Please select the device:
1. Set device
2. Custom device
Enter the number corresponding to your device: 1


In [311]:
#Custom Parameters

tapered = False
taperEnd = 0
taperStart = 0


# L0 = 33
# C0 = 1332
# RADB = 16
# RADA = 11.55
# Z0 = 60
# R0 = 6.3
# V0 = 27
# P0 = 3.5
# MW = 4
# ZN = 1
# dissociatenumber = 2

# massf = 0.14
# CURRF = 0.7
# massfr = 0.35
# currfr = 0.7

In [312]:
# Constants
Mu = 1.257 * 10 ** -6
Pi = 3.142
bc = 1.38 * 10 ** -23
mi = 1.67 * 10 ** -27
MUK = Mu / (8 * Pi * Pi * bc)
CON11 = 1.6 * 10 ** -20
CON12 = 9.999999 * 10 ** -21
CON2 = 4.6 * 10 ** -31
UGCONS = 8.310001 * 10 ** 3
FRF = 0.3
fe = 1 / 3
FLAG = 0

EINP = 0

In [313]:
# If operating in Deuterium, set value of G
if ZN == 1 or ZN == 2:
    g = 1.66667
else:
    # If Ne or argon or Xenon, Kr or N2, set approx initial values of G
    g = 1.3

G1 = 2 / (g + 1)
G2 = (g - 1) / g
GCAP = (g + 1) / (g - 1)

# Calculate ambient number density and some ratios
N0 = 2.69 * (10 ** 25) * P0 / 760
C = RADB / RADA
f = Z0 / RADA

# Convert to SI units
C0 = C0 * 10 ** -6
L0 = L0 * 10 ** -9
RADB = RADB * 0.01
RADA = RADA * 0.01
Z0 = Z0 * 10 ** -2
V0 = V0 * 1000
RHO = P0 * 2.33 * (10 ** -4) * MW / 4
TM = 0

In [314]:
# Vary g value based on whether not using D-T or Hydrogen based gases
# Constants

if tapered:
    taperStart = taperStart * 0.01
    taperEnd = taperEnd * 0.01
    zTaperStart = taperStart / Z0
    taperGrad = (RADA - taperEnd) / (Z0 - taperStart)

In [318]:
# Calculate characteristic quantities and scaling parameters

I0 = V0 / math.sqrt(L0 / C0)
T0 = math.sqrt(L0 * C0)
TA = math.sqrt(4 * Pi * Pi * (C * C - 1) / (Mu * math.log(C))) * ((Z0 * math.sqrt(RHO)) / (I0 / RADA)) * ((math.sqrt(massf)) / CURRF)



ZZCHAR = Z0 / TA
AL = T0 / TA
AA = math.sqrt((g + 1) * (C * C - 1) / math.log(C)) * (f / 2) * (math.sqrt(massf / massfr)) * (currfr / CURRF)

RESF = R0 / (math.sqrt(L0 / C0))
BE = 2 * (10 ** -7) * math.log(C) * Z0 * CURRF / L0
BF = BE / (math.log(C) * f)

VPINCHCH = ZZCHAR * AA / f
TPINCHCH = RADA / VPINCHCH

# Calculate ratio of characteristic capacitor time to sum of characteristic axial & radial times
ALT = (AL * AA) / (1 + AA)
print(AL)


9.282415568840161


In [319]:
# Corresponding to VBA Line 415
print(f"RHO: {RHO}, I0: {I0}, T0: {T0}, TA: {TA}")

# Corresponding to VBA Line 419
print(f"ZZCHAR: {ZZCHAR}, VPINCHCH: {VPINCHCH}, TPINCHCH: {TPINCHCH}")

# Corresponding to VBA Line 422
print(f"C: {C}, f: {f}, AL: {AL}, AA: {AA}, BE: {BE}, RESF: {RESF}, ALT: {ALT}")

RHO: 0.00023300000000000003, I0: 4472135.954999579, T0: 2.6832815729997478e-06, TA: 2.890714763953435e-07
ZZCHAR: 1037805.6103664489, VPINCHCH: 796691.0053182985, TPINCHCH: 9.413938339875642e-08
C: 1.6666666666666667, f: 4.0, AL: 9.282415568840161, AA: 3.0706752685100134, BE: 0.35757793663619336, RESF: 670.8203932499368, ALT: 7.002102117986637


In [320]:
# Applying checks on pressure and atomic number
if P0 > 20:
    raise Exception("Pressure above 20 torr will not produce focussing. Reduce fill pressure below 20 torr.")
if ZN in [1, 2] and P0 <= 0.1:
    raise Exception("In real experiments, Pressure below 0.1 torr does not yield good focus. Increase fill pressure above 0.1 torr.")
elif ZN in [7, 10, 18, 36, 54] and P0 <= 0.05:
    raise Exception("In real experiments in neon or argon or xenon, Pressure below 0.05 torr may not yield good focus. Increase fill pressure above 0.05 torr.")
elif ZN not in [1, 2, 7, 10, 18, 36, 54]:
    raise Exception("Invalid Atomic number.")

# Check for atomic values and ratio of characteristic capacitor time to sum of characteristic axial & radial times, stop the program if not valid
if ZN in [1, 2]:
    if ALT <= 0.68:
        raise Exception("Total TRANSIT TIME (axial + radial) MAY BE TOO LONG COMPARED TO effective DISCHARGE Drive TIME for atomic numbers 1 and 2")
elif ZN in [7, 10, 18, 36, 54]:
    if ALT <= 0.65:
        raise Exception("Total TRANSIT TIME (axial + radial) MAY BE TOO LONG COMPARED TO effective DISCHARGE Drive TIME for atomic numbers 7, 10, 18, 36, 54")      
else:
    raise Exception("Invalid Atomic number")



# 1.0 Axial Phase

In [321]:
# Record data from numerical integration
rowj = 20

# Set time increment and initial values
D = 0.002
T = 0
Z = 0
ZZ = 0
II = 1
I = 0
IO = 0
AC = AL * math.sqrt(1 / 2)
V=0
Wpiston = 0 # TODO find correct Wpiston value
zeff = 0.000000328 #TODO Only accurate for Ne right now, needs to me updated using the Corona models https://www.plasmafocus.net/IPFS/modelpackage/Corona%20Calculations/C1coronaIntroduction.htm
Ipeak = 0


In [322]:
# Start numerical integration of AXIAL PHASE

df = pd.DataFrame(columns=['TR', 'IR', 'VR', 'ZR', 'ZZR', 'IR_CURRF', 'Lz_10_9', 'Einductance', 'EINP', 'ni_10_23', 'TM_10_6', 'nimax_10_23', 'Wpiston', 'T'])

while Z < 1 and T < 6:

    T += D
    ZZ += AC * D
    Z += ZZ * D
    I += II * D
    IO += I * D

    # Unit converstion to real, but convenient units
    TR = T * T0 * 10**6
    VR = V * V0 * 10**-3
    IR = I * I0 * 10**-3
    ZZR = (ZZCHAR / AL) * ZZ * 10**-4
    ACR = ((ZZCHAR / AL) / T0) * AC * 10**-10
    ZR = Z * Z0 * 100
    IIR = (II * I0 / T0) * 10**-9
    IOR = IO * I0 * T0


    if IR > Ipeak:
        Ipeak = IR

    DR = D * T0 # DR is time increment in sec
    DZR = DR * ZZR * 10 ** 4 # DZR is z increment in m

    # Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
    EINP += (10 ** -7) * (math.log(C) * ZZR * (10 ** 4) * IR * IR * (10 ** 6) * CURRF * CURRF) * DR

    # Integrate for piston work
    Wpiston += (10 ** -7) * (math.log(C)) * IR * IR * (10 ** 6) * CURRF * CURRF * DZR
    Lz = (10 ** -9) * ZR * 2 * math.log(C)

    # Einductance is total inductive energy (in all inductances)
    Einductance = 0.5 * 10 ** 6 * IR * IR * (CURRF * CURRF * Lz + L0)

    ni = massf * N0 * ((g + 1) / (g - 1))
    nimax = N0 * ((g + 1) / (g - 1))

    # Following statement computes shock temperature, however need to first enumerate zeff
    TM = 2.4 * (10 ** -4) * ((g - 1) / ((g + 1) ** 2)) * (MW / zeff) * ZZR * (10 ** 8)

    # Append a new row to the DataFrame
    df = df.append({
        'TR': TR,
        'IR': IR,
        'VR': VR,
        'ZR': ZR,
        'ZZR': ZZR,
        'IR_CURRF': IR * CURRF,
        'Lz_10_9': Lz * 10 ** 9,
        'Einductance': 100 * Einductance / (0.5 * C0 * V0 * V0),
        'EINP': 100 * EINP / (0.5 * C0 * V0 * V0),
        'ni_10_23': ni / (10 ** 23),
        'TM_10_6': TM / (10 ** 6),
        'nimax_10_23': nimax / (10 ** 23),
        'Wpiston': 100 * Wpiston / (0.5 * C0 * V0 * V0),
        'T': T
    }, ignore_index=True)


    # If no taper
    if tapered == 0 or Z < zTAPERSTART:
        V = Z * II + ZZ * I
        V *= BE
        AC = (AL * AL * I * I - ZZ * ZZ) / Z
        II = (1 - IO - BE * ZZ * I - RESF * I) / (1 + BE * Z)
    else:
        # Compute tapered anode for this axial position
        w = f * tapergrad * (Z - zTAPERSTART)
        c1 = RADB / (RADA * (1 - w))
        TA = math.sqrt(4 * math.pi * math.pi * (C * C - 1) / (Mu * math.log(c1))) * ((Z0 * math.sqrt(RHO)) / (I0 / RADA)) * ((math.sqrt(massf)) / CURRF)
        AL1 = T0 / TA
        BE = 2 * (10 ** -7) * math.log(C) * Z0 * CURRF / L0
        tc1 = 1 + (w * (4 - 3 * w)) / (2 * (C * C - 1))
        tc2 = w * (Z - zTAPERSTART) * (2 - w) / (2 * (C * C - 1))
        LOG1 = math.log(1 / (1 - w))
        tc3 = 1 + LOG1 / math.log(C)
        tc4 = Z + ((1 / math.log(C)) * (1 - ((1 - w) / w) * LOG1) * (Z - zTAPERSTART))

        V = (tc4) * II + ZZ * I * tc3
        V *= BE
        AC = (AL1 * AL1 * I * I - ZZ * ZZ * tc1) / (Z + tc2)
        II = (1 - IO - BE * tc3 * ZZ * I - RESF * I) / (1 + BE * tc4)

        
print(df)

             TR        IR        VR        ZR       ZZR  IR_CURRF   Lz_10_9  \
0      0.005367  8.944272  0.000000  0.000788  0.146768  6.260990  0.000805   
1      0.010733  5.888453  0.001878  0.002363  0.293536  4.121917  0.002414   
2      0.016100  6.932337  0.000274  0.003116  0.140254  4.852636  0.003183   
3      0.021466  6.575767  0.001129  0.003926  0.150948  4.603037  0.004011   
4      0.026833  6.697539  0.000523  0.004739  0.151633  4.688277  0.004842   
...         ...       ...       ...       ...       ...       ...       ...   
2996  16.083590  6.607325  0.000720  2.481371  0.153789  4.625128  2.535096   
2997  16.088956  6.607306  0.000720  2.482196  0.153788  4.625114  2.535939   
2998  16.094323  6.607286  0.000720  2.483022  0.153788  4.625100  2.536782   
2999  16.099689  6.607266  0.000720  2.483847  0.153788  4.625086  2.537625   
3000  16.105056  6.607246  0.000720  2.484672  0.153787  4.625072  2.538468   

      Einductance          EINP  ni_10_23      TM_1

In [323]:
print(df.iloc[1000:1050])

            TR        IR        VR        ZR       ZZR  IR_CURRF   Lz_10_9  \
1000  5.371930  6.646761  0.000732  0.830776  0.154399  4.652733  0.848764   
1001  5.377296  6.646742  0.000732  0.831605  0.154398  4.652719  0.849610   
1002  5.382663  6.646722  0.000732  0.832434  0.154398  4.652705  0.850457   
1003  5.388029  6.646702  0.000732  0.833262  0.154398  4.652691  0.851303   
1004  5.393396  6.646682  0.000732  0.834091  0.154397  4.652677  0.852150   
1005  5.398763  6.646662  0.000732  0.834919  0.154397  4.652664  0.852996   
1006  5.404129  6.646642  0.000732  0.835748  0.154397  4.652650  0.853843   
1007  5.409496  6.646623  0.000732  0.836577  0.154397  4.652636  0.854689   
1008  5.414862  6.646603  0.000732  0.837405  0.154396  4.652622  0.855536   
1009  5.420229  6.646583  0.000732  0.838234  0.154396  4.652608  0.856382   
1010  5.425595  6.646563  0.000732  0.839062  0.154396  4.652594  0.857229   
1011  5.430962  6.646543  0.000732  0.839891  0.154395  4.652580

In [226]:
Laxial = ZR * 2 * math.log(C) # TODO figure out what this is for
ZG = ZZ
CFA = CURRF
# Introduce differential in current factors for axial and radial phases
CurrentFactorRatio = currfr / CURRF
CFR = CurrentFactorRatio

BE *= CFR
BF *= CFR
CURRF *= CFR
AAg = AA / math.sqrt(g + 1)

if tapered:
    w1 = f * tapergrad * (1 - zTAPERSTART)
    c2 = RADB / (RADA * (1 - w1))
    AA1 = AA * (RADA / ENDRAD) * math.sqrt((math.log(C) / math.log(c2)))
    LOG2 = math.log(1 / (1 - w1))
    tc5 = 1 + ((1 / math.log(C)) * (1 - ((1 - w1) / w1) * LOG2) * (1 - zTAPERSTART))
    AAg1 = AA1 / math.sqrt(g + 1)

# Radial phase RI, distances are relative to radius a.
# KS is shock position, KP is radial piston position, ZF is focus
# pinch length, all normalized to inner radius a; VS and VP are
# radial shock and piston speed,VZ is axial pinch length elongating rate
# Distances, radius and speeds are relative to radius of anode.
# AS BEFORE QUANTITIES WITH AN R ATTACHED HAVE BEEN RE-COMPUTED AS REAL, i.e. UN-NORMALIZED QUANTITIES EXPRESSED IN USUAL LABORATORY UNITS.

# End of Axial Phase; Start of Radial Inward Shock Phase

# 2.0 Radial Inward Shock Phase

In [229]:
# Set some initial values for Radial Inward Phase step-by-step integration
# Reset time increment to finer step-size
KS = 1
KP = 1python 
ZF = 1E-05
zFLAG = 0
gFLAG = 0

print(ZF)

1e-05


In [228]:
while KS > 0:

    # Set time increment to have about 1500 (up to 3000 for high pressure) steps in Radial Inward Shock Phase
    if tapered:
        TPINCHCH *= (ENDRAD / RADA) * (ENDRAD / RADA)

    DREAL = TPINCHCH / 700
    D = DREAL / T0

    # Set initial 'LookBack' values, for compensation of finite small disturbance speed
    IDELAY = I
    KPDELAY = KP
    KSDELAY = KS
    VSDELAY = -1

    # Set initial value, approximately, for CHARGE NUMBER Z
    # For H2,D2 and He, assume fully ionized with gamma=1.667 during all of radial phase
    Z = ZN

    # Integrate radial inward shock phase in non-dimentional units

    # Calculate inward shock speed
    trradialstart = T * T0
    # Assuming ActiveSheet.Cells(11, 8) corresponds to a variable in Python
    variable_11_8 = trradialstart * (10 ** 6)

    if not tapered:
        GCAP = (g + 1) / (g - 1)
        AA = AAg * math.sqrt(g + 1)
        VS = -AL * AA * IDELAY / (KPDELAY)
        VSR = VS * RADA / T0
    else:
        AA1 = AAg1 * math.sqrt(g + 1)
        VS = -AL1 * AA1 * IDELAY / (KPDELAY)
        VSR = VS * ENDRAD / T0

    # Real temperature is needed to DETERMINE SMALL DISTURBANCE SPEED FOR COMMUNICATION TIME CORRECTION.
    # Hence the shock speed is re-calculated in SI units, then Plasma Temp TM is calculated, based on shock theory
    TM = (MW / (8315)) * ((GCAP - 1) / (GCAP * GCAP)) * ((VSR * VSR) / ((1 + Z) * dissociatenumber))
    TeV = TM / (1.16 * 10 ** 4)
    
    if ZN == 1 or ZN == 2:
        G1 = 2 / (g + 1)
        G2 = (g - 1) / g
    elif ZN == 7:
        g, Z, G1, G2, zFLAG, gFLAG = nitrogen_corona(TeV, zFLAG, gFLAG, FLAG)
    elif ZN == 10:
        g, Z = neon_corona(TM, FLAG)
    elif ZN == 18:
        g, Z, G1, G2 = argon_corona(TM, FLAG)
    elif ZN == 36:
        g, Z, G1, G2, zFLAG, gFLAG = krypton_corona(TeV, zFLAG, gFLAG, FLAG)
    elif ZN == 54:
        g, Z, G1, G2 = xenon_corona(TM, FLAG)
        
    VZ = -G1 * VS
    K1 = KS / KP

    E1 = G1 * K1 * VSDELAY
    E2 = (1 / g) * (KP / I) * (1 - K1 * K1) * II
    E3 = (G1 / 2) * (KP / ZF) * (1 - K1 * K1) * VZ

    E4 = G2 + (1 / g) * K1 * K1
    VP = (E1 - E2 - E3) / E4

    if tapered == False:
        V = (BE - BF * math.log(KP / C) * ZF) * II - I * (BF * (ZF / KP) * VP + (BF * math.log(KP / C)) * VZ)
        II = (1 - IO + BF * I * (ZF / KP) * VP + BF * math.log(KP / C) * I * VZ - RESF * I) / (1 + BE - BF * math.log(KP / C) * ZF)
    else:
        V = (BE * tc5 - BF * math.log(KP / c2) * ZF) * II - I * (BF * (ZF / KP) * VP + (BF * math.log(KP / c2)) * VZ)
        II = (1 - IO + BF * I * (ZF / KP) * VP + BF * math.log(KP / c2) * I * VZ - RESF * I) / (1 + BE * tc5 - BF * math.log(KP / c2) * ZF)

    # Increment time and Integrate, by linear approx, for I, flowed charge I0, KS, KP and ZF
    T = T + D
    I = I + II * D
    IO = IO + I * D
    KS = KS + VS * D
    KP = KP + VP * D
    ZF = ZF + VZ * D

    if tapered == False:
        SSR = VS * (RADA / T0) * 10 ** -4
        SPR = VP * (RADA / T0) * 10 ** -4
        SZR = VZ * (RADA / T0) * 10 ** -4
        KSR = KS * RADA * 1000
        kpr = KP * RADA * 1000
        zfr = ZF * RADA * 1000
    else:
        SSR = VS * (ENDRAD / T0) * 10 ** -4
        SPR = VP * (ENDRAD / T0) * 10 ** -4
        SZR = VZ * (ENDRAD / T0) * 10 ** -4
        KSR = KS * ENDRAD * 1000
        kpr = KP * ENDRAD * 1000
        zfr = ZF * ENDRAD * 1000

    TR = T * T0 * 10 ** 6
    VR = V * V0 * 10 ** -3
    IR = I * I0 * 10 ** -3
    IIR = (II * I0 / T0) * 10 ** -9

    # DR is time increment in secs, DKPR is piston position increment & DZFR length position increment, both in SI units
    DR = D * T0
    DKPR = SPR * DR * 10 ** 4
    DZFR = SZR * DR * 10 ** 4

    if IR > Ipeak:
        Ipeak = IR

    # Obtain Max induced voltage
    if VR > VRMAX:
        VRMAX = VR

    TRRadial = TR * (10 ** 3) - trradialstart * (10 ** 9)

    if SSR < peakvs:
        peakvs = SSR

    if SPR < peakvp:
        peakvp = SPR

    # Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
    EINP = EINP + (10 ** 3) * (SZR * math.log(1000 * RADB / kpr) - (SPR * (zfr / kpr))) * IR * IR * CURRF * CURRF * DR

    # Also integrate for piston work
    Wpiston = Wpiston + 0.1 * (DZFR * math.log(1000 * RADB / kpr) - DKPR * (zfr / kpr)) * IR * IR * CURRF * CURRF
    Lplasma = Laxial + (zfr * 2 * (10 ** -1) * math.log(1000 * RADB / kpr))

    Einductance = 0.5 * 10 ** 6 * IR * IR * (CURRF * CURRF * Lplasma * (10 ** -9) + L0)
    ni = massfr * N0 * ((g + 1) / (g - 1))
    nimax = N0 * ((g + 1) / (g - 1))

    df = df.append({
        'TR': TR,
        'IR': IR,
        'VR': VR,
        'ZR': zfr,
        'ZZR': SZR,
        'IR_CURRF': IR * CURRF,
        'Lz_10_9': Lplasma,
        'Einductance': 100 * Einductance / (0.5 * C0 * V0 * V0),
        'EINP': 100 * EINP / (0.5 * C0 * V0 * V0),
        'ni_10_23': ni / (10 ** 23),
        'TM_10_6': TM / (10 ** 6),
        'nimax_10_23': nimax / (10 ** 23),
        'Wpiston': 100 * Wpiston / (0.5 * C0 * V0 * V0),
        'T': TRRadial
    }, ignore_index=True)

    if KSR > kpr:
        SDSPEED = math.sqrt(g * dissociatenumber * (1 + Z) * bc * TM / (MW * mi))
        SDDELAYTIME = ((kpr - KSR) / 1000) / SDSPEED
        backhowmanyrows = SDDELAYTIME / DR
        BACKROWNUMBER = rowi - backhowmanyrows
        if BACKROWNUMBER < FIRSTRADIALROW:
            BACKROWNUMBER = FIRSTRADIALROW + 1
        delayrow = BACKROWNUMBER - 1
        if delayrow < 20:
            delayrow = 20


        # TODO: Missing IDELAY and VSDELAY writing to final df

    else:
        IDELAY = I
        KPDELAY = KP
        VSDELAY = VS
        KSDELAY = KS


             TR        IR        VR        ZR       ZZR  IR_CURRF   Lz_10_9  \
2999  16.099689  8.240617  0.001119  3.102378  0.191974  5.768432  3.169548   
3000  16.105056  8.240586  0.001119  3.103408  0.191973  5.768410  3.170601   
3001  16.105190  8.240586  0.000584  0.000898  0.110116  5.768411  3.170693   
3002  16.105325  8.240587  0.000669  0.001046  0.110120  5.768411  3.170708   
3003  16.105459  8.240587  0.000668  0.001194  0.110124  5.768411  3.170723   
...         ...       ...       ...       ...       ...       ...       ...   
3494  16.171483  8.240248  0.000619  0.074313  0.111363  5.768174  3.178361   
3495  16.171617  8.240247  0.000619  0.074462  0.111365  5.768173  3.178377   
3496  16.171752  8.240246  0.000619  0.074612  0.111368  5.768172  3.178393   
3497  16.171886  8.240246  0.000619  0.074762  0.111370  5.768172  3.178409   
3498  16.172021  8.240245  0.000620  0.074912  0.111373  5.768171  3.178425   

      Einductance      EINP  ni_10_23      TM_10_6 

In [238]:
print(df.iloc[30:100])

          TR        IR        VR        ZR       ZZR  IR_CURRF   Lz_10_9  \
30  0.166363  8.332301  0.001152  0.032328  0.193356  5.832611  0.033028   
31  0.171730  8.332270  0.001152  0.033365  0.193358  5.832589  0.034088   
32  0.177097  8.332239  0.001152  0.034403  0.193360  5.832568  0.035148   
33  0.182463  8.332208  0.001152  0.035441  0.193361  5.832546  0.036208   
34  0.187830  8.332177  0.001152  0.036478  0.193362  5.832524  0.037268   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.515190  8.330283  0.001152  0.099779  0.193357  5.831198  0.101939   
96  0.520557  8.330252  0.001152  0.100816  0.193357  5.831177  0.102999   
97  0.525923  8.330221  0.001151  0.101854  0.193357  5.831155  0.104059   
98  0.531290  8.330190  0.001151  0.102892  0.193356  5.831133  0.105119   
99  0.536656  8.330159  0.001151  0.103929  0.193356  5.831111  0.106179   

    Einductance      EINP  ni_10_23      TM_10_6  nimax_10_23       Wpiston  \
30     0

# 3.0 

In [174]:
RRF = 0
FRF = 0.33
VSV = VS
VRF = -FRF * VS
G1 = 2 / (g + 1)
G2 = (g - 1) / g
MUP = Mu / (2 * math.pi)
VZ = -G1 * VS
T = T + D
 
RRF = RRF + VRF * D
VRFCMUS = VRF * 10 ** -4 
K1 = 0
E1 = G1 * K1 * VSV
E2 = (1 / g) * (rp / I) * (1 - K1 * K1) * IDOT 
E3 = (G1 / 2) * (rp / ZF) * (1 - K1 * K1) * VZ
E4 = G2 + (1 / g) * K1 * K1
VP = (E1 - E2 - E3) / E4 
IDOT = (V0 - (CH / C0) - I * R0 - I * CURRF * MUP * ((math.log(RADB / rp)) * VZ - (ZF / rp) * VP)) / (L0 + MUP * CURRF * ((math.log(C)) * Z0 * tc5 + (math.lo (RADB / rp)) * ZF))
V = MUP * I * ((math.log(RADB / rp)) * VZ - (ZF / rp) * VP) + MUP * ((math.lo (RADB / rp)) * ZF + (math.log(C)) * Z0 * tc5) * IDOT 
V = V * CURRF
I = I + IDOT * D
CH = CH + I * D
rp = rp + VP * D
ZF = ZF + VZ * D
 
DKPR = SPR * D * 10 ** 4 
if IR > Ipeak:
     Ipeak = IR 
# Obtain Max induced voltage
VR = V * 10 ** -3 
if VR > VRMAX:
    VRMAX = VR
    TRRadial = TR * 10 ** 3 - trradialstart * 10 ** 9 
if SSR < peakvs:
    peakvs = SSR 
if SPR < peakvp:
    peakvp = SPR 
# Convert to Real convenient units for print out
TR = T * 10 ** 6
VR = V * 10 ** -3
IR = I * 10 ** -3
kpr = rp * 10 ** 3
zfr = ZF * 10 ** 3
SPR = VP * 10 ** -4 
SZR = VZ * 10 ** -4
IDOTKAUS = IDOT * 10 ** -9
RRFMM = RRF * 10 ** 3
TRRadial = TR * 10 ** 3 - trradialstart * 10 ** 9 
# Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
EINP = EINP + (10 ** -7) * (SZR * 10 ** 4 * math.log(1000 * RADB / kpr) - (SPR * 10 ** 4 * (1000 / kpr) * (zfr / 1000))) * IR * IR * 10 ** 6 * CURRF * CURRF * D 
# Also integrate for piston work
Wpiston = Wpiston + 0.1 * (DZFR * math.log(1000 * RADB / kpr) - DKPR * (zfr / kpr)) * IR * IR * CURRF * CURRF 
if IR > Ipeak:
    Ipeak = IR 
# Determine max induced voltage for beam-gas neutron yield computation
if VR > VRMAX:
    VRMAX = VR
    Lplasma = Laxial + (zfr * 2 * 10 ** -1 * math.log(1000 * RADB / kpr))
    Einductance = 0.5 * 10 ** 6 * IR * IR * (CURRF * CURRF * Lplasma * 10 ** -9 + L0) 
# Use Dratio from RS of strong shocks as described above
ni = ni1 * Dratio
nimax = nimax1 * Dratio
new_row = {'TR': TR, 'IR': IR, 'VR': VR, 'ZR': ZR, 'ZZR': ZZR, 'IR_CURRF': IR * CURRF, 'Lz_10_9': Lplasma, 
    'Einductance': 100 * Einductance / (0.5 * C0 * V0 * V0), 'EINP': 100 * EINP / (0.5 * C0 * V0 * V0), 
    'ni_10_23': ni / (10 ** 23), 'TM_10_6': TMRS / (10 ** 6), 'nimax_10_23': nimax / (10 ** 23), 
    'Wpiston': 100 * Wpiston / (0.5 * C0 * V0 * V0), 'T': T} 
df.loc[len(df)] = new_row

NameError: name 'rp' is not defined